# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [2]:
import pandas as pd
import requests

# 1) Há linhas duplicadas?
df = pd.read_csv('/content/SINASC_RO_2019.csv')
print(df.shape)
df.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [3]:
# 2) Conte o número de valores missing por variável.
df.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [4]:
# 3) Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes.
#    Então crie uma seleção dessa base somente com as colunas que interessam.

dados_sel = df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

dados_sel.isna().sum().sort_values(ascending = False)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [5]:
dados_sel

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [6]:
# 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração.
#Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal.
#Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.

dados_sel['APGAR5'].shape #verificando quantidade de linhas do campo

(27028,)

In [7]:
dados_sel['APGAR5'].isna().sum() #verificando os valores faltantes

103

In [15]:
dados_sel = dados_sel.dropna(subset = ['APGAR5']) #removendo os registros

In [16]:
dados_sel['APGAR5'].isna().sum() #verificando se os registros foram removidos

0

In [17]:
# 5) Observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado.
#   Vamos assumir que o não preenchido é o mesmo que o código 9.

dados_sel['ESTCIVMAE'].value_counts().sort_index()

ESTCIVMAE
1.0    8907
2.0    9906
3.0      57
4.0     451
5.0    7145
9.0     144
Name: count, dtype: int64

In [18]:
dados_sel['CONSULTAS'].value_counts().sort_index()

CONSULTAS
1      972
2     1359
3     5957
4    18587
9       50
Name: count, dtype: int64

In [19]:
dados_sel['ESTCIVMAE'].isna().sum() #verificando os valores faltantes do campo

315

In [24]:
dados_sel['ESTCIVMAE'].fillna(9,inplace = True) #preenchendo os campos faltantes com 9
dados_sel['ESTCIVMAE'].isna().sum()

0

In [25]:
dados_sel['CONSULTAS'].isna().sum() #verificando que o campo 'Consultas' nao possui dados faltantes

0

In [26]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero

dados_sel['QTDFILVIVO'].isna().sum() #verificando a quantidade de dados faltantes


1566

In [31]:
dados_sel['QTDFILVIVO'].fillna(0,inplace = True) #substituindo os campos faltantes por 0
dados_sel['QTDFILVIVO'].isna().sum() #sem dados faltantes

0

In [33]:
# 7) Das restantes, decida que valores te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha.

dados_sel[['LOCNASC', 'IDADEMAE', 'ESCMAE','GESTACAO', 'GRAVIDEZ']].isna().sum() #vericando quais colunas possuem dados faltantes.

LOCNASC        0
IDADEMAE       0
ESCMAE       310
GESTACAO    1216
GRAVIDEZ      76
dtype: int64

Como pode-se observar, as colunas 'LOCNASC'e 'IDADEMAE'não possui dados faltantes. Então não será necessario realizar tratamento. Começando pela escolaridade da mãe temos:

In [34]:
dados_sel['ESCMAE'].value_counts() #verificando o nivel de escolaridade das maes

ESCMAE
8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: count, dtype: int64

In [35]:
dados_sel['ESCMAE'].isna().sum() #verificando a quantidade de dados faltantes da coluna

310

Como a escolaridade da mãe não é um fator predominante, vou preencher os dados faltantes com 'Ignorado', mostrando que foi um campo que nao foi preenchido.

In [37]:
dados_sel['ESCMAE'].fillna('Ignorado',inplace = True)
dados_sel['ESCMAE'].value_counts()


<ipython-input-37-ef417b700405>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_sel['ESCMAE'].fillna('Ignorado',inplace = True)


ESCMAE
8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Ignorado             310
Nenhuma               40
Name: count, dtype: int64

Proximo coluna a ser estudada será a da 'Gravidez'

In [38]:
dados_sel['GRAVIDEZ'].value_counts() #verificando a quantidade de dados presentes na coluna

GRAVIDEZ
Única              26356
Dupla                488
Tríplice e mais        5
Name: count, dtype: int64

In [39]:
dados_sel['GRAVIDEZ'].isna().sum() #verificando a quantidade de dados faltantes na coluna

76

O tipo de gravidez já é uma variavel bem sensivel, uma vez que a quantidade de bebes que nascem em uma gravidez sao em sua maioria única, poucas duplas e menos ainda tiplas ou mais. Sendo assim acredito ser viavel retirar esses dados faltantes da tabela para nao comprometer o restante.

In [40]:
dados_sel = dados_sel.dropna(subset = ['GRAVIDEZ']) #removendo os registros
dados_sel['GRAVIDEZ'].isna().sum() #verificando se os registros foram removidos

0

Agora para ultima coluna de interesse a ser analisada 'GESTACAO'

In [41]:
dados_sel['GESTACAO'].value_counts() #verificando a quantidade de dados presentes na coluna

GESTACAO
37 a 41 semanas        22471
32 a 36 semanas         2226
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: count, dtype: int64

In [42]:
dados_sel['GESTACAO'].isna().sum() #verificando a quantidade de dados faltantes na coluna

1200

No caso da 'GESTACAO'optei por comprentar os valores faltantes utilizando o metodo bfill. Preenchendo o dado faltante com o dado anterior

In [44]:
dados_sel['GESTACAO'].bfill(inplace = True)
verific = dados_sel['GESTACAO'].isna().sum() #verificando se os registros foram removidos
print('Existem',verific,'dados faltantes')
dados_sel['GESTACAO'].value_counts()

Existem 0 dados faltantes


GESTACAO
37 a 41 semanas        23490
32 a 36 semanas         2359
42 semanas e mais        730
28 a 31 semanas          168
22 a 27 semanas           95
Menos de 22 semanas        7
Name: count, dtype: int64

8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  


In [45]:
#8) selecionando os dados da coluna APGAR5

dados_sel['APGAR5']


0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26849, dtype: float64

In [46]:
dados_sel['APGAR5'].value_counts().sort_index()

APGAR5
0.0        17
1.0        24
2.0        18
3.0        15
4.0        23
5.0        45
6.0        79
7.0       240
8.0      1984
9.0     16035
10.0     8369
Name: count, dtype: int64

É possivel observar que em sua grande maioria o apgar se encontra em sua faixa normal que é entre 8 e 10.

In [47]:
dados_sel.loc[dados_sel['APGAR5'] >= 8, 'apgar5_new'] = 'normal'
dados_sel.loc[(dados_sel['APGAR5'] >=6) & (dados_sel['APGAR5'] <= 7),'apgar5_new'] = 'asfixia leve'
dados_sel.loc[(dados_sel['APGAR5'] >= 4) & (df['APGAR5'] <= 5),'apgar5_new'] = 'asfixia moderada'
dados_sel.loc[(dados_sel['APGAR5'] <= 3), 'apgar5_new'] = 'asfixia severa'

dados_sel['apgar5_new'].value_counts() #exibe a frequencia

apgar5_new
normal              26388
asfixia leve          319
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [48]:
# 9)  Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras.
#     Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

dados_sel.rename(columns = lambda x: x.lower().replace(" ","_"),inplace =True)
dados_sel

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar5_new
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
